First, let's import our packages. I struggled with Beautiful Soup in the first instance, so let's just use the read_html function.

In [1]:
import pandas as pd
import numpy as np

Then, we can extract the information from the wikipedia page, and view the first five rows. 

In [2]:
tor_neighborhoodss = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)
tor_neighborhoods = tor_neighborhoodss[0]
tor_neighborhoods.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


Then, let's change the headings to get rid of the first column name, which is likely to cause problems. 

In [3]:
tor_neighborhoods.columns = ['PostalCode', 'Borough', 'Neighborhood']
tor_neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


The next step is removing those which are unassigned. First, let's check how many we have.

In [4]:
tor_neighborhoods[tor_neighborhoods.Borough == 'Not assigned']['Neighborhood'].unique()
print("There are {} records where the borough is not assigned".format(
    tor_neighborhoods[tor_neighborhoods.Borough == 'Not assigned'].shape[0]))

There are 77 records where the borough is not assigned


Then we remove those with a simple overwrite of the tor neighborhoods object. 

In [5]:
tor_neighborhoods = tor_neighborhoods[tor_neighborhoods.Borough != 'Not assigned']

Then, let's merge the neighborhoods where the postcodes and borough's are the same.

In [6]:
tor_neighborhoods = (tor_neighborhoods.groupby(['PostalCode', 'Borough'])['Neighborhood']
      .apply(lambda x: "{}".format(', '.join(x))).reset_index())

Now, we can make the neighborhoods the same as the boroughs in the contexts where the neighborhood is not assigned.

In [7]:
tor_neighborhoods.loc[tor_neighborhoods.Neighborhood.str.contains('Not assigned'), 'Neighborhood'] = \
    tor_neighborhoods.loc[tor_neighborhoods.Neighborhood.str.contains('Not assigned'), 'Borough']

We can check this is correct by searching for non assigned neighborhoods.

In [8]:
tor_neighborhoods[tor_neighborhoods.Neighborhood.str.contains('Not assigned')]

,PostalCode,Borough,Neighborhood


As we can see, there are now none. Finally, we're going to look at the shape of the dataframe we've created.

In [9]:
tor_neighborhoods.shape

(103, 3)